# SynPlanner tutorial: from reaction data to retrosynthetic planning

<font size="4">This tutorial explains how to perform the retrosynthetic models training in SynPlanner</font>

**Introduction.** Welcome to the SynPlanner tutorial, a detailed guide on utilizing a sophisticated retrosynthetic planning tool that combines Monte Carlo Tree Search (MCTS) with neural networks. This tutorial is designed for synthetic chemists and chemoinformaticians looking to deepen their understanding and application of SynPlanner in their work.

**About SynPlanner.** SynPlanner is aimed as a one-box solution for building retrosynthetic planners and includes the original modules for reaction data curation, reaction rules extraction, retrosynthetic models training, and retrosynthetic planning. SynPlanner is based on the open-source Python 3 packages for reaction chemoinformatics and deep learning frameworks.

**Tutorial focus.** This tutorial will lead through the steps of extraction of reaction rules, training retrosynthetic models (ranking policy network), and retrosynthetic planning in SynPlanner.

## Before starting

There are two reaction datasets versions:

1) Full reaction dataset (around 1280K reactions) – for preparation of real retrosynthetic planner
2) Reduced reaction dataset (72K reactions) – for demonstrative purposes.

For educational, demonstrative purposes and on machines with limited computational resources (CPU and RAM) it is recommended to use reduced reaction datasets. The full reaction dataset practically is executable now only on computational servers.

## 1. Download input data

The input data will be downloaded from the Google disk to the current location (./). The training data directory includes:

 - `uspto_standardized.smi` - the reaction dataset for reaction rules extraction and ranking policy network.
 - `building_blocks.smi` -  a set of building block molecules used in value neural network tuning (not considered in this tutorial) and in retrosynthetic planning.
 - `molecules_for_filtering_policy_training.smi` - a set of molecules for filtering policy network training (not considered in this tutorial)
 - `targets_for_value_network_training.smi` - a set of target molecules for planning simulations in value network tuning (not considered in this tutorial).

In [1]:
from synplan.utils.loading import download_all_data

In [2]:
download_all_data(save_to=".")

Downloading...
From (original): https://drive.google.com/uc?id=1I2u4Zn-et-tlHmdLBAqclGiAiUuA1X5G
From (redirected): https://drive.google.com/uc?id=1I2u4Zn-et-tlHmdLBAqclGiAiUuA1X5G&confirm=t&uuid=6c51c349-5a5e-4f28-86e3-7badad2d990d
To: /home1/dima/synplanner/tutorials/synplanner_training_data.zip
100%|██████████████████| 128M/128M [00:01<00:00, 84.9MB/s]


## 2. Set up input data and working firectory

For the default training and planning pipeline in SynPlanner only two datasets are required ``reaction dataset`` and ``building blocks set``.

In [3]:
import pickle
from pathlib import Path

In [4]:
root_folder = Path("training_results") # replace this path where you want to keep all your results
root_folder.mkdir(exist_ok=True)

In [5]:
REACTION_DATA_PATH = "uspto_standardized_reduced.smi"
BUILDING_BLOCKS_PATH = "building_blocks.smi"

## 3. Reaction data standardization

**IMPORTANT**: In this tutorial, the input data are already standardized by a slightly different protocol from the paper [https://doi.org/10.1002/minf.202100119]. 
Here, the standardization of this data is performed for demonstration. Skip this step if needed.

The reaction data are standardized using an original protocol for reaction data curation published earlier [https://doi.org/10.1002/minf.202100119]. This protocol includes two layers: standardization of individual molecules (reactants, reagents, products), reaction atom mapping and reaction standardization. Steps for standardization of individual molecules include functional group standardization, aromatization/kekulization, valence checking, hydrogens manipulation, cleaning isotopes, and radicals, etc. The reaction standardization layer includes reaction role assignment, reaction equation balancing, and atom-to-atom mapping fixing. The duplicate reactions and erroneous reactions are removed.

Currently available standardizers in SynPlanner:

1. **ReactionMappingStandardizer**: Maps atoms of the reaction using chython and chytorch.
2. **FunctionalGroupsStandardizer**: Standardization of functional groups.
3. **KekuleFormStandardizer**: Transforms molecules to Kekule form when possible.
4. **CheckValenceStandardizer**: Checks atom valences.
5. **ImplicifyHydrogensStandardizer**: Removes hydrogen atoms.
6. **CheckIsotopesStandardizer**: Checks and clean isotope atoms when possible.
7. **SplitIonsStandardizer**: Splits ions in reaction when possible.
8. **AromaticFormStandardizer**: Transforms molecules to aromatic form when possible.
9. **MappingFixStandardizer**: Fixes atom-to-atom mapping in reaction when needed and possible.
10. **UnchangedPartsStandardizer**: Removes unchanged parts in reaction.
11. **DuplicateReactionStandardizer**: Removes duplicate reactions.

In [6]:
from synplan.chem.data.standardizing import (
    ReactionStandardizationConfig,
    standardize_reactions_from_file,
)

from synplan.chem.data.standardizing import (
    ReactionMappingStandardizer,
    FunctionalGroupsConfig,
    KekuleFormConfig,
    CheckValenceConfig,
    ImplicifyHydrogensConfig,
    CheckIsotopesConfig,
    AromaticFormConfig,
    MappingFixConfig,
    UnchangedPartsConfig,
    DuplicateReactionConfig,
)

### Setting up the reaction standardization configuration

The next step is to configure the reaction standardization process. We do this using the `ReactionStandardizationConfig` class in SynPlanner. This class allows for the specification of various parameters and settings for the standardization process.

**Important**: if the reaction standardizer name is listed in the configuration file or in `ReactionStandardizationConfig`, it means that this standardizer will be activated.

In [7]:
standardization_config = ReactionStandardizationConfig(
    reaction_mapping_config=ReactionStandardizationConfig(),
    functional_groups_config=FunctionalGroupsConfig(),
    kekule_form_config=KekuleFormConfig(),
    check_valence_config=CheckValenceConfig(),
    implicify_hydrogens_config=ImplicifyHydrogensConfig(),
    check_isotopes_config=CheckIsotopesConfig(),
    aromatic_form_config=AromaticFormConfig(),
    mapping_fix_config=MappingFixConfig(),
    unchanged_parts_config=UnchangedPartsConfig(),
    duplicate_reaction_config=DuplicateReactionConfig(),
)

Once this standardization configuration is in place, we can proceed to apply these standardozers to our reaction data:

In [8]:
standardized_data_path = root_folder.joinpath(
    "uspto_standardized.smi"
)  # standardized reaction data file

standardize_reactions_from_file(
    config=standardization_config,
    input_reaction_data_path=REACTION_DATA_PATH,
    standardized_reaction_data_path=standardized_data_path,
    num_cpus=4,
    batch_size=100,
)

Number of reactions processed: 71832 [08:45]


Initial number of parsed reactions: 71832
Standardized number of reactions: 69446


## 4. Reaction data filtration

In SynPlanner, reaction data filtration is a crucial step to ensure the quality and accuracy of the data used for retrosynthetic analysis. The USPTO dataset, a standardized but unfiltered collection of reaction records, serves as the primary data source. However, this dataset may contain records with no reaction center or atom-to-atom mapping errors. To address this, we apply several reaction filters:

1. **NoReactionFilter**: Removes reactions with identical reactants and products.
2. **SmallMoleculesFilter**: Excludes reactions where all reactants and products have ≤ specified number of heavy atoms.
3. **DynamicBondsFilter**: Discards reactions with more than specified changed bonds.
4. **MultiCenterFilter**: Eliminates records with multiple reaction centers.
5. **CCsp3BreakingFilter**: Removes reactions where a bond between two sp3 carbons is broken, indicating potential mapping errors.
6. **CCRingBreakingFilter**: Filters out reactions breaking a bond between two carbons in the same ring (sizes 5, 6, or 7), identified using the SSSR algorithm.
7. **WrongCHBreakingFilter**: Removes reactions breaking a C-H bond to form a C-C bond, with exceptions for condensation reactions or those involving carbenes.

First, we need to import the main config class called `ReactionCheckConfig`, which will encapsulate the list of reaction filters. Finally, we call the `filter_reactions` for filtering the input reaction data.

In [9]:
from synplan.chem.data.filtering import (
    ReactionFilterConfig,  # the main config class
    filter_reactions_from_file,  # reaction filtration function
    # reaction filters:
    CCRingBreakingConfig,
    WrongCHBreakingConfig,
    CCsp3BreakingConfig,
    DynamicBondsConfig,
    MultiCenterConfig,
    NoReactionConfig,
    SmallMoleculesConfig,
)

### Setting up the reaction filtration configuration

The next step is to configure the reaction filtration process. We do this using the `ReactionCheckConfig` class in SynPlanner. This class allows for the specification of various parameters and settings for the filtration process.

**Important**: if the reaction filter name is listed in the configuration file or in `ReactionCheckConfig`, it means that this filter will be activated. Also, some filters requires additional parameters (e.g. `small_molecules_config`).

In [10]:
filtration_config = ReactionFilterConfig(
    dynamic_bonds_config=DynamicBondsConfig(
        min_bonds_number=1,  # minimum number of dynamic bonds for a reaction
        max_bonds_number=6,
    ),  # maximum number of dynamic bonds for a reaction
    no_reaction_config=NoReactionConfig(),  # configuration for the 'no reaction' filter.
    multi_center_config=MultiCenterConfig(),  # configuration for the 'multi-center reaction' filter
    wrong_ch_breaking_config=WrongCHBreakingConfig(),  # configuration for the 'C-H breaking' filter
    cc_sp3_breaking_config=CCsp3BreakingConfig(),  # configuration for the 'Csp3-C breaking' filter
    cc_ring_breaking_config=CCRingBreakingConfig(),
)  # configuration for the 'C-C ring breaking' filter

By setting up `filtration_config`, we are essentially telling SynPlanner what filters to apply and how to apply them. This step is crucial for ensuring that the data we use for further analysis is as accurate and reliable as possible. The reaction filters we apply here are based on the specific needs of our analysis and the characteristics of the USPTO dataset.

Once this configuration is in place, we can proceed to apply these filters to our reaction data:

In [11]:
filtered_data_path = root_folder.joinpath(
    "uspto_filtered.smi"
)  # filtered reaction data file

filter_reactions_from_file(
    config=filtration_config,
    input_reaction_data_path=standardized_data_path,  # the path to the standardized reaction data file
    filtered_reaction_data_path=filtered_data_path,  # the path to the filtered reaction data file
    num_cpus=4,
    batch_size=100,
)

Number of reactions processed: 69446 [04:45]


Initial number of reactions: 69446
Removed number of reactions: 1


## 5. Reaction rules extraction

After filtering the reaction data, the next step in SynPlanner training pipeline is the extraction of reaction rules. The reaction rules define the transformations that can be applied to synthesize target molecules.

### The extraction protocol
The protocol for extracting reaction rules from reactions in SynPlanner utilizes the CGRtools Python library. This procedure involves the following steps:

**1. CGR creation**

The input reaction is converted to the Condensed Graph of Reaction (CGR), which is a single graph encoding an ensemble of reactants and products. CGR results from the superposition of the atoms of products and reactants having the same numbers. It contains both conventional chemical bonds (single, double, triple, aromatic, etc.) and so-called “dynamic” bonds describing chemical transformations, i.e. breaking or forming a bond or changing bond order. Given CGRs it is possible to extract the reaction center of the reaction.

**2. Reaction center extraction**

Some reactions can have several reaction centers and if the multicenter_rules parameter is True, these centers will be included in a single reaction rule. Otherwise, each reaction center will be included in different reaction rules.

**3. Reaction center extension**

The extracted reaction center can be extended by the inclusion of neighboring atoms (environmental atoms) of radius N (can be specified by environment_atom_count parameter). Usually, the environment of radius 1 is included.

**4. Ring structures inclusion**

If the reaction center atoms are part of ring structures or functional groups, they can be included in the reaction center. Ring structures are identified by the Smallest Set of Smallest Rings (SSSR) algorithm implemented in CGRTools. If the include_rings parameter is True and any atom of the reaction center is part of the identified ring structure, the whole ring is included in the reaction center.

**5. Functional group inclusion**

Functional groups to be included in the reaction center should be specified manually as a list of their SMILES in func_groups_list in the configuration file. If include_func_groups is True, any atom of the reaction center is part of any specified functional group, and the whole functional group is included in the reaction center.

**6. Leaving/incoming groups inclusion**

Leaving and incoming groups can be included in the reaction center. These groups are identified by comparison of the atoms in reactants and products. If some atoms in reactants are not observed in products, these atoms are identified as a leaving group. Likewise, if some atoms in products are not observed in reactants, these atoms are identified as an incoming group. This functionality is supposed to handle protection/deprotection reactions and identify protective agents. Leaving groups are added to the reaction center if the keep_leaving_groups parameter is True and incoming groups are added to the reaction center if keep_incoming_groups parameter is True.

**7. Reagents inclusion**

Reaction rule can be further specified by the inclusion of reagents if the keep_reagents parameter is True. It means that structurally identical reaction rules with identical atom properties will be considered different if they are associated with different reagents.

**8. Atom properties specification**

Each atom in the extended and specified reaction center and its environment atoms (neighbor atoms) is described by four properties: the number of neighbors, hybridization type, the number of hydrogens, and the size of the ring (if the atom belongs to any ring). These properties determine the level of atom and reaction rule specification and can be disabled if needed in the atom_info_retention section in the configuration file. If some property in atom_info_retention is True, it means that this property of the atom will be included in the final reaction rule.

**9. Reaction rule creation**

After all settings, the final reaction rule is assembled, reversed (to be used in retrosynthetic mode), and validated by application of the final reaction rule to the original reaction, from which it was extracted.

**10. Reaction rule validation**

Finally, the extracted rules are filtered by popularity, which is defined by the min_popularity parameter. For example, min_popularity:3 means, that only rules observed in not less than 3 reactions from the reaction dataset are remained.

### Configuring reaction rule extraction
The `ExtractRuleConfig` class in SynPlanner allows for the fine-tuning of how reaction rules are extracted. Key parameters of this class include:

- **multicenter_rules**: Determines whether a single rule is extracted for all centers in multicenter reactions (`True`) or if separate rules are generated for each center (`False`).
- **include_rings**: Includes ring structures in the reaction rules connected to the reaction center atoms if set to `True`.
- **keep_leaving_groups**: Keeps the leaving groups in the extracted reaction rule when set to `True`.
- **keep_incoming_groups**: Retains incoming groups in the extracted reaction rule if set to `True`.
- **keep_reagents**: Includes reagents in the extracted reaction rule when `True`. Default is `False`.
- **environment_atom_count**: Sets the number of layers of atoms around the reaction center to be included in the rule. A value of `0` includes only the reaction center, `1` includes the first surrounding layer, and so on.
- **min_popularity**: Establishes the minimum number of times a rule must be applied to be included in further analysis.
- **atom_info_retention**: Dictates the level of detail retained about atoms in the reaction center and their environment. Default settings retain information about neighbors, hybridization, implicit hydrogens, and ring sizes for both the reaction center and its environment. Default settings:
    ```python
    
    {
        "reaction_center": {
            "neighbors": True,
            "hybridization": True,
            "implicit_hydrogens": False,
            "ring_sizes": False,
        },
        "environment": {
            "neighbors": False,
            "hybridization": False,
            "implicit_hydrogens": False,
            "ring_sizes": False,
        },
    }
    ```

These settings are crucial for tailoring the reaction rule extraction process to the specific needs of the retrosynthetic analysis, ensuring that the rules are both accurate and relevant.

In [12]:
from synplan.utils.config import RuleExtractionConfig
from synplan.chem.reaction_rules.extraction import extract_rules_from_reactions

In [13]:
extraction_config = RuleExtractionConfig(
    min_popularity=3,
    environment_atom_count=1,
    multicenter_rules=True,
    include_rings=False,
    keep_leaving_groups=True,
    keep_incoming_groups=False,
    keep_reagents=False,
    atom_info_retention={
        "reaction_center": {
            "neighbors": True,  # retains information about neighboring atoms to the reaction center
            "hybridization": True,  # preserves the hybridization state of atoms at the reaction center
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to the reaction center
            "ring_sizes": False,  # keeps information about the sizes of rings that reaction center atoms are part of
        },
        "environment": {
            "neighbors": False,  # retains information about neighboring atoms to the atoms in the environment of the reaction center
            "hybridization": False,  # preserves the hybridization state of atoms in the environment
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to atoms in the environment
            "ring_sizes": False,  # keeps information about the sizes of rings that environment atoms are part of
        },
    },
)

This setup, for instance, retains leaving groups and specifies the level of detail for atom information retention around the reaction center. Each parameter in `ExtractRuleConfig` is designed to give users the flexibility to optimize rule extraction according to their requirements.

### Extracting reaction rules using `extract_rules_from_reactions`

After configuring the rule extraction settings in SynPlanner, the next step is to apply these configurations to extract reaction rules from the reaction data. This is achieved using the `extract_rules_from_reactions` function.

In [14]:
reaction_rules_path = root_folder.joinpath("reaction_rules.pickle")  # extracted reaction rules file

extract_rules_from_reactions(
    config=extraction_config,  # the configuration settings for rule extraction
    reaction_data_path=filtered_data_path,  # path to the reaction data file
    reaction_rules_path=reaction_rules_path,  # path to the pickle file where the extracted reaction rules will be stored
    num_cpus=4,
    batch_size=100,
)

Number of reactions processed: 69445 [04:10]


Number of extracted reaction rules: 21880


The extracted reaction rules can be loaded and visually inspected (the reaction rules are sorted by popularity):

In [15]:
with open(reaction_rules_path, "rb") as f:
    reaction_rules_list = [i[0] for i in pickle.load(f)]
for rule in reaction_rules_list:
    rule.clean2d()

In [16]:
reaction_rules_list[0]

In [17]:
reaction_rules_list[1]

In [18]:
reaction_rules_list[100]

## 6. Ranking policy training

After extracting the reaction rules, the next step in SynPlanner involves the training ranking policy network. 

The tree nodes in MCTS are expanded by an expansion function approximated by a policy graph neural network. The policy network is composed of two parts: molecular representation and reaction rule prediction parts. In the representation part, the molecular graph is converted to a single vector by graph convolutional layers. The training set structure and the prediction part architecture depend on the type of policy network, particularly the ranking or filtering policy network. The training dataset for ranking policy network consists of pairs of reactions and corresponding reaction rules extracted from it. The products of the reaction are transformed to the CGR encoded as a molecular graph with the one-hot encoded label vector where the positive label corresponds to the reaction rule. The prediction part is terminated with the softmax function generating the “probability of successful application” of each reaction rule to a given input molecular graph, which can be used for the reaction rules “ranking”.
All the reaction rules predicted by the ranking or policy neural network are sorted by predicted reaction rule probability and the first N (usually N = 50) of reaction rules are selected to be applied to the current precursor in the expansion step.

Frst, we define the training configuration using the `PolicyNetworkConfig` class. This configuration includes various hyperparameters for the neural network:

In [19]:
from synplan.utils.config import PolicyNetworkConfig
from synplan.ml.training.supervised import create_policy_dataset, run_policy_training

In [20]:
training_config = PolicyNetworkConfig(
    policy_type="ranking",  # the type of policy network
    num_conv_layers=5,  # the number of graph convolutional layers in the network
    vector_dim=512,  # the dimensionality of the final embedding vector
    learning_rate=0.0008,  # the learning rate for the training process
    dropout=0.4,  # the dropout rate
    num_epoch=100,  # the number of epochs for training
    batch_size=100,
)  # the size of training batch of input data

#### Creating the training set

Next, we create the policy dataset using the `create_policy_dataset` function. This involves specifying paths to the reaction rules and the reaction data:

In [21]:
ranking_policy_network_folder = root_folder.joinpath("ranking_policy_network")
ranking_policy_dataset_path = ranking_policy_network_folder.joinpath("ranking_policy_dataset.pt")

datamodule = create_policy_dataset(
    dataset_type="ranking",
    reaction_rules_path=reaction_rules_path,
    molecules_or_reactions_path=filtered_data_path,
    output_path=ranking_policy_dataset_path,
    batch_size=training_config.batch_size,
    num_cpus=4,
)

Number of reactions processed: 69445 [11:33]


Training set size: 52509, validation set size: 13128


#### Run the policy network training

Finally, we train the policy network using the `run_policy_training` function. This step involves feeding the dataset and the training configuration into the network:

In [22]:
run_policy_training(
    datamodule,  # the prepared data module for training
    config=training_config,  # the training configuration
    results_path=ranking_policy_network_folder,
)  # path to save the training results

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Epoch 00012: reducing learning rate of group 0 to 6.4000e-04.
Epoch 00016: reducing learning rate of group 0 to 5.1200e-04.
Epoch 00020: reducing learning rate of group 0 to 4.0960e-04.
Epoch 00024: reducing learning rate of group 0 to 3.2768e-04.
Epoch 00028: reducing learning rate of group 0 to 2.6214e-04.
Epoch 00032: reducing learning rate of group 0 to 2.0972e-04.
Epoch 00036: reducing learning rate of group 0 to 1.6777e-04.
Epoch 00040: reducing learning rate of group 0 to 1.3422e-04.
Epoch 00044: reducing learning rate of group 0 to 1.0737e-04.
Epoch 00048: reducing learning rate of group 0 to 8.5899e-05.
Epoch 00052: reducing learning rate of group 0 to 6.8719e-05.
Epoch 00056: reducing learning rate of group 0 to 5.4976e-05.
Epoch 00060: reducing learning rate of group 0 to 5.0000e-05.
Policy network balanced accuracy: 0.985


## 7. Tree search with the ranking policy network and rollout evaluation

This section explained how to perform retrosynthetic planning with extracted reaction rules, trained ranking policy network, and MCTS algorithm with rollout node evaluation.

### Configuring the tree search
The tree search is configured using the `TreeConfig` class. Key parameters include:

- **max_iterations**: Defines the total number of iterations the algorithm will perform, essentially setting a limit on how many times the tree search loop can run.
- **max_tree_size**: Sets the upper limit on the total number of nodes that can exist in the search tree, controlling the size and complexity of the tree.
- **max_time**: Specifies a time limit for the algorithm's execution, measured in seconds. This prevents the search from running indefinitely.
- **max_depth**: Determines the maximum depth of the tree, effectively controlling how far the search can go from the root node. The default maximum depth is 6 levels.
- **ucb_type**: Chooses the type of Upper Confidence Bound (UCB) algorithm used in the search. Options include "puct" (predictive UCB), "uct" (standard UCB), and "value".
- **c_ucb**: This is the exploration-exploitation balance coefficient in UCB, which influences how much the algorithm favors exploration of new paths versus exploitation of known paths.
- **backprop_type**: Selects the backpropagation method used during the search. Options are "muzero" (model-based approach) and "cumulative" (cumulative reward approach).
- **search_strategy**: Determines the strategy for navigating the tree. Options are "expansion_first" (prioritizing the expansion of new nodes) and "evaluation_first" (prioritizing the evaluation of existing nodes).
- **exclude_small**: A boolean setting that, when true, excludes small molecules from the search, typically to focus on more complex molecules.
- **evaluation_agg**: This setting determines how the evaluation scores are aggregated. Options are "max" (using the maximum score) and "average" (using the average score).
- **evaluation_type**: Defines the method used for node evaluation. Options include "random" (random evaluations), "rollout" (using rollout simulations), and "gcn" (graph convolutional networks).
- **init_node_value**: Sets the initial value for newly created nodes in the tree. This can impact how nodes are prioritized during the search.
- **epsilon**: This parameter is used in the epsilon-greedy strategy during node selection, representing the probability of choosing a random action for exploration. A higher value leads to more exploration.
- **min_mol_size**: Defines the minimum size of a molecule (in terms of the number of heavy atoms) to be considered in the search. Molecules smaller than this threshold are typically considered as readily available building blocks.
- **silent**: When set to True, this option suppresses the progress output of the tree search, keeping the output clean and focused.

In [23]:
from CGRtools import smiles

from IPython.display import SVG, display

from synplan.utils.visualisation import get_route_svg
from synplan.mcts.tree import Tree
from synplan.mcts.expansion import PolicyNetworkFunction
from synplan.utils.config import TreeConfig
from synplan.utils.loading import load_reaction_rules, load_building_blocks

In [24]:
tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_type="rollout",
    max_iterations=300,
    max_time=120,
    max_depth=9,
    min_mol_size=0,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

### Choose the target molecule

In [25]:
example_molecule = "CC(C)(C)[Si](OCc1ccn[nH]c1=O)(c1ccccc1)c1ccccc1"

target = smiles(example_molecule)
target.canonicalize()
target.clean2d()
target

### Initializing the tree and running search

The `Tree` class is initialized with the target molecule, the path to reaction rules, building blocks, the tree configuration, and the policy function. The policy function, obtained from the trained ranking policy, guides the selection of retrosynthetic transformations.

In [26]:
# initialize the policy function
ranking_policy_network = ranking_policy_network_folder.joinpath("policy_network.ckpt")
policy_config = PolicyNetworkConfig(weights_path=ranking_policy_network)
policy_function = PolicyNetworkFunction(policy_config=policy_config)

reaction_rules = load_reaction_rules(reaction_rules_path)
building_blocks = load_building_blocks(BUILDING_BLOCKS_PATH)

In [27]:
tree = Tree(
    target=target,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_function,
    evaluation_function=None,
)

In [28]:
tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True
tree

  0%|          | 0/300 [00:00<?, ?it/s]

Tree for: c1ccccc1[Si](c2ccccc2)(OCC=3C(=O)NN=CC=3)C(C)(C)C
Time: 5.7 seconds
Number of nodes: 169
Number of iterations: 300
Number of visited nodes: 70
Number of found routes: 16

### Retrosynthetic routes visualisation

After the tree search is complete, we can visualize the found retrosynthetic paths. The visualization uses the `get_route_svg` function from SynPlanner visualization interface.

In [29]:
for n, node_id in enumerate(tree.winning_nodes):
    print(
        f"-------- Path starts from node #{node_id} with total route score {tree.route_score(node_id)} --------"
    )
    display(SVG(get_route_svg(tree, node_id)))
    if n == 3:
        break

-------- Path starts from node #5 with total route score -0.06818117454094759 --------


-------- Path starts from node #46 with total route score 0.012269499202705397 --------


-------- Path starts from node #46 with total route score 0.012269499202705397 --------


-------- Path starts from node #63 with total route score 0.015060834021355295 --------
